<a href="https://colab.research.google.com/github/fataik1/DS-Unit-1-Sprint-2-Statistics/blob/master/FataiKingDS_14_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [1]:
### YOUR CODE STARTS HERE
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-03-16 19:21:48--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2020-03-16 19:21:48 (614 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import ttest_1samp, ttest_ind, ttest_ind_from_stats, ttest_rel

df = pd.read_csv('house-votes-84.data', names = ['party', 'handicapped-infants', 'water-project', 'budget',
                                                 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 'anti-satellite-ban',
                                                 'aid-to-contras', 'mx-missle', 'immigration', 'synfuels', 
                                                 'education', 'right-to-sue' ,'crime', 'duty-free', 'south-africa'])

df.head()                 
                 

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missle,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [3]:
#Clean Data
df.dtypes

party                   object
handicapped-infants     object
water-project           object
budget                  object
physician-fee-freeze    object
el-salvador-aid         object
religious-groups        object
anti-satellite-ban      object
aid-to-contras          object
mx-missle               object
immigration             object
synfuels                object
education               object
right-to-sue            object
crime                   object
duty-free               object
south-africa            object
dtype: object

In [5]:
#Change NO & Yes to 0 & 1
df = df.replace({"y": 1, "n": 0, "?": np.NaN})

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missle,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [7]:
#Shorter name
rep = df[df['party']=='republican']

rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missle,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [8]:
#Shorter name
dem = df[df['party']=='democrat']

dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missle,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [24]:
dem.shape

(267, 17)

In [25]:
rep.shape

(168, 17)

In [29]:
#Create a function
# a = sample one, b = null hypothesis, omit nan values
def one_sample_ttest(a,b,nan_policy='omit'):
  return ttest_1samp(a,b,nan_policy='omit')

# a = sample one, b = sample 2, omit nan values
def two_sample_ttest(a,b,nan_policy='omit'):
  return ttest_ind(a,b,nan_policy='omit')

one_sample_ttest(rep['physician-fee-freeze'], 1)
two_sample_ttest(rep['handicapped-infants'], dem['handicapped-infants'])

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

In [32]:
# create a list for all the features for rep and dem data
# Makes life easy

rep_data = [rep['handicapped-infants'], rep['physician-fee-freeze'], 
           rep['water-project'], rep['budget'],
           rep['el-salvador-aid'], rep['religious-groups'],
           rep['anti-satellite-ban'], rep['aid-to-contras'],
           rep['mx-missle'], rep['immigration'],
           rep['synfuels'], rep['education'],
           rep['right-to-sue'], rep['crime'],
           rep['duty-free'], rep['south-africa']]
           

dem_data = [dem['handicapped-infants'], dem['physician-fee-freeze'], 
           dem['water-project'], dem['budget'],
           dem['el-salvador-aid'], dem['religious-groups'],
           dem['anti-satellite-ban'], dem['aid-to-contras'],
           dem['mx-missle'], dem['immigration'],
           dem['synfuels'], dem['education'],
           dem['right-to-sue'], dem['crime'],
           dem['duty-free'], dem['south-africa']]

# test the list
# republican data with a null hypothesis of 0.5
[one_sample_ttest(a,0.5) for a in rep_data]



[Ttest_1sampResult(statistic=-10.232833482397659, pvalue=2.572179359890009e-19),
 Ttest_1sampResult(statistic=57.09643655679979, pvalue=3.8884659739684535e-110),
 Ttest_1sampResult(statistic=0.16385760607458383, pvalue=0.8700683158522193),
 Ttest_1sampResult(statistic=-13.705331355148527, pvalue=6.249973784238298e-29),
 Ttest_1sampResult(statistic=26.920515819250607, pvalue=4.531756691646779e-62),
 Ttest_1sampResult(statistic=16.844895175868118, pvalue=1.103043623086e-37),
 Ttest_1sampResult(statistic=-7.694446231812848, pvalue=1.3430544790393879e-12),
 Ttest_1sampResult(statistic=-12.048344034968558, pvalue=4.758689364006887e-24),
 Ttest_1sampResult(statistic=-15.439826683525418, pvalue=8.750571156437711e-34),
 Ttest_1sampResult(statistic=1.4845341263724807, pvalue=0.1395867786115413),
 Ttest_1sampResult(statistic=-13.659513100277254, pvalue=1.5157657361710158e-28),
 Ttest_1sampResult(statistic=13.7323961384641, pvalue=1.5754742985301713e-28),
 Ttest_1sampResult(statistic=13.057014526

In [38]:
#Dem data with a null hypothesis of 0.5
[one_sample_ttest(a,0.5) for a in dem_data]


[Ttest_1sampResult(statistic=3.431373087696574, pvalue=0.000699612317167372),
 Ttest_1sampResult(statistic=-31.67705343439813, pvalue=6.796885728494356e-91),
 Ttest_1sampResult(statistic=0.06454972243678961, pvalue=0.9485867005339235),
 Ttest_1sampResult(statistic=19.859406568628835, pvalue=5.75931504660857e-54),
 Ttest_1sampResult(statistic=-11.016877548066462, pvalue=2.5007537432253433e-23),
 Ttest_1sampResult(statistic=-0.7464459604122172, pvalue=0.45608033540995874),
 Ttest_1sampResult(statistic=10.424565592705058, pvalue=1.8326900884510166e-21),
 Ttest_1sampResult(statistic=14.13618595353591, pvalue=4.190313037098042e-34),
 Ttest_1sampResult(statistic=9.470521640429526, pvalue=2.3590277159598606e-18),
 Ttest_1sampResult(statistic=-0.9246846884388146, pvalue=0.35598081794452796),
 Ttest_1sampResult(statistic=0.18751153526716705, pvalue=0.8514092819865365),
 Ttest_1sampResult(statistic=-15.915784824279635, pvalue=8.900198902856228e-40),
 Ttest_1sampResult(statistic=-7.34555787391736

In [33]:
# republican data with a null hypothesis of 1
[one_sample_ttest(a,1) for a in rep_data]

[Ttest_1sampResult(statistic=-26.625236633811387, pvalue=1.978873197183477e-61),
 Ttest_1sampResult(statistic=-1.4185450076223511, pvalue=0.1579292482594923),
 Ttest_1sampResult(statistic=-11.961605243444543, pvalue=1.8656648229239887e-23),
 Ttest_1sampResult(statistic=-32.43595087385152, pvalue=5.293293090366981e-73),
 Ttest_1sampResult(statistic=-2.890793645020198, pvalue=0.004363402589282088),
 Ttest_1sampResult(statistic=-4.338836636208457, pvalue=2.488389920449274e-05),
 Ttest_1sampResult(statistic=-22.533735393166197, pvalue=1.1791229999687983e-51),
 Ttest_1sampResult(statistic=-29.402380855978315, pvalue=1.646323339608132e-65),
 Ttest_1sampResult(statistic=-35.49944402826317, pvalue=6.985438779792963e-79),
 Ttest_1sampResult(statistic=-11.407472760546423, pvalue=1.4608897556395605e-22),
 Ttest_1sampResult(statistic=-32.22244115962839, pvalue=2.3835416257429457e-71),
 Ttest_1sampResult(statistic=-4.776485613378817, pvalue=4.122594240061259e-06),
 Ttest_1sampResult(statistic=-5.03

In [34]:
# democrat data with a null hypothesis of 1
[[one_sample_ttest(a,1) for a in dem_data]]

[[Ttest_1sampResult(statistic=-12.96296499796484, pvalue=6.590394568934029e-30),
  Ttest_1sampResult(statistic=-67.19374970932937, pvalue=1.7479453896049469e-165),
  Ttest_1sampResult(statistic=-15.36283393995609, pvalue=1.8031537722768159e-37),
  Ttest_1sampResult(statistic=-5.702205846437985, pvalue=3.217258173105712e-08),
  Ttest_1sampResult(statistic=-30.39138633949369, pvalue=1.4023105444477201e-86),
  Ttest_1sampResult(statistic=-16.7950341092749, pvalue=3.12862805821349e-43),
  Ttest_1sampResult(statistic=-8.724104538575864, pvalue=3.377871549159516e-16),
  Ttest_1sampResult(statistic=-7.354085178140069, pvalue=2.464384519854037e-12),
  Ttest_1sampResult(statistic=-8.87861403790268, pvalue=1.418109388321789e-16),
  Ttest_1sampResult(statistic=-17.137489559066015, pvalue=1.1309092327824875e-44),
  Ttest_1sampResult(statistic=-15.750968962442089, pvalue=1.839716417353916e-39),
  Ttest_1sampResult(statistic=-38.305787204198445, pvalue=3.89658572232871e-106),
  Ttest_1sampResult(sta

# An issue Democrats support more than Republicans with p < 0.01 is their take on Physician-Fee-Freeze. Dems Pvalue is 0.000699612317167372.

# An issue that republicans support more than democrats with p < 0.01 is Republicans take on South Africa. The P value for that is 0.00010154897627277522

# An issue where the difference between republicans and democrats has p > 0.1 is their takes on aid-to-contras. Reps had a pvalue of 5.293 while, Dems Pvalue was 3.21. 2 will be the pvalue which is greater than 0.1

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!